In [2]:
# https://github.com/alisaaalehi/convolution_as_multiplication/blob/master/Convolution_as_multiplication.ipynb

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time
from PIL import Image
import cv2

from scipy.linalg import toeplitz
from keras import backend as K
from tensorflow import keras
from tensorflow.keras.preprocessing import image
print(tf.__version__)

2.1.0


In [44]:
# input signal
I = np.arange(16).reshape(4,4)
print('I: ', I.shape)
print(I)

 # filter 
F = np.array([1,-1,1,0,-1,0,0,1,0]).reshape(3,3)
print('F: ',F.shape)
print(F)

I:  (4, 4)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
F:  (3, 3)
[[ 1 -1  1]
 [ 0 -1  0]
 [ 0  1  0]]


In [ ]:
# input signal
I = np.array([[1, 2, 3], [4, 5, 6]])
print('I: ', I.shape)
print(I)

 # filter 
F = np.array([[10, 20], [30, 40]])
print('F: ',F.shape)
print(F)

In [45]:
# number columns and rows of the input 
I_row_num, I_col_num = I.shape 

# number of columns and rows of the filter
F_row_num, F_col_num = F.shape

#  calculate the output dimensions
output_row_num = I_row_num + F_row_num - 1
output_col_num = I_col_num + F_col_num - 1
print('output dimension:', output_row_num, output_col_num)

output dimension: 6 6


In [46]:
# zero pad the filter
F_zero_padded = np.pad(F, ((output_row_num - F_row_num, 0),
                           (0, output_col_num - F_col_num)),
                        'constant', constant_values=0)
print('F_zero_padded: ', F_zero_padded)

F_zero_padded:  [[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1  0  0  0]
 [ 0 -1  0  0  0  0]
 [ 0  1  0  0  0  0]]


In [47]:
from scipy.linalg import toeplitz

# use each row of the zero-padded F to creat a toeplitz matrix. 
#  Number of columns in this matrices are same as numbe of columns of input signal
toeplitz_list = []
for i in range(F_zero_padded.shape[0]-1, -1, -1): # iterate from last row to the first row
    c = F_zero_padded[i, :] # i th row of the F 
    r = np.r_[c[0], np.zeros(I_col_num-1)] # first row for the toeplitz fuction should be defined otherwise
                                                        # the result is wrong
    toeplitz_m = toeplitz(c,r) # this function is in scipy.linalg library
    toeplitz_list.append(toeplitz_m)
    print('F '+ str(i)+'\n', toeplitz_m)

F 5
 [[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
F 4
 [[ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0.  0.]]
F 3
 [[ 1.  0.  0.  0.]
 [-1.  1.  0.  0.]
 [ 1. -1.  1.  0.]
 [ 0.  1. -1.  1.]
 [ 0.  0.  1. -1.]
 [ 0.  0.  0.  1.]]
F 2
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
F 1
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
F 0
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [48]:
# doubly blocked toeplitz indices: 
#  this matrix defines which toeplitz matrix from toeplitz_list goes to which part of the doubly blocked
c = range(1, F_zero_padded.shape[0]+1)
r = np.r_[c[0], np.zeros(I_row_num-1, dtype=int)]
doubly_indices = toeplitz(c, r)
print('doubly indices \n', doubly_indices)

doubly indices 
 [[1 0 0 0]
 [2 1 0 0]
 [3 2 1 0]
 [4 3 2 1]
 [5 4 3 2]
 [6 5 4 3]]


In [49]:
## creat doubly blocked matrix with zero values
toeplitz_shape = toeplitz_list[0].shape # shape of one toeplitz matrix
h = toeplitz_shape[0]*doubly_indices.shape[0]
w = toeplitz_shape[1]*doubly_indices.shape[1]
doubly_blocked_shape = [h, w]
doubly_blocked = np.zeros(doubly_blocked_shape)

In [50]:
# tile toeplitz matrices for each row in the doubly blocked matrix
b_h, b_w = toeplitz_shape # hight and withs of each block
for i in range(doubly_indices.shape[0]):
    for j in range(doubly_indices.shape[1]):
        start_i = i * b_h
        start_j = j * b_w
        end_i = start_i + b_h
        end_j = start_j + b_w
        doubly_blocked[start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1]

print('doubly_blocked: ', doubly_blocked)

doubly_blocked:  [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0. -1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -1.  1.  0.  0. -1.  0.  0.  0.  1.  0.

In [51]:
def matrix_to_vector(input):
    input_h, input_w = input.shape
    output_vector = np.zeros(input_h*input_w, dtype=input.dtype)
    # flip the input matrix up-down because last row should go first
    input = np.flipud(input) 
    for i,row in enumerate(input):
        st = i*input_w
        nd = st + input_w
        output_vector[st:nd] = row
        
    return output_vector

In [52]:
# call the function
vectorized_I = matrix_to_vector(I)
print('vectorized_I: ', vectorized_I)

vectorized_I:  [12 13 14 15  8  9 10 11  4  5  6  7  0  1  2  3]


In [53]:
# get result of the convolution by matrix mupltiplication
result_vector = np.matmul(doubly_blocked, vectorized_I)
print('result_vector: ', result_vector)

result_vector:  [ 0. 12. 13. 14. 15.  0.  0. -4. -4. -4. -4.  0. 12. -3.  9. 10. -5. 15.
  8. -3.  5.  6. -5. 11.  4.  1.  4.  4. -4.  7.  0.  1.  1.  2. -1.  3.]


In [54]:
def vector_to_matrix(input, output_shape):
    output_h, output_w = output_shape
    output = np.zeros(output_shape, dtype=input.dtype)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[i, :] = input[st:nd]
    # flip the output matrix up-down to get correct result
    output=np.flipud(output)
    return output

In [55]:
# reshape the raw rsult to desired matrix form
out_shape = [output_row_num, output_col_num]
my_output = vector_to_matrix(result_vector, out_shape)

print('Result of implemented method: \n', my_output)

Result of implemented method: 
 [[ 0.  1.  1.  2. -1.  3.]
 [ 4.  1.  4.  4. -4.  7.]
 [ 8. -3.  5.  6. -5. 11.]
 [12. -3.  9. 10. -5. 15.]
 [ 0. -4. -4. -4. -4.  0.]
 [ 0. 12. 13. 14. 15.  0.]]


In [60]:
from scipy import signal

lib_output = signal.convolve2d(I, F, "full")

In [61]:
lib_output

array([[ 0,  1,  1,  2, -1,  3],
       [ 4,  1,  4,  4, -4,  7],
       [ 8, -3,  5,  6, -5, 11],
       [12, -3,  9, 10, -5, 15],
       [ 0, -4, -4, -4, -4,  0],
       [ 0, 12, 13, 14, 15,  0]])